<a href="https://colab.research.google.com/github/dfreire770/dq-ddq-smb-agent/blob/main/Mario_Video_Reward_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install gym==0.26.2
!pip install gym-notices==0.0.8
!pip install gymnasium==0.29.1
!pip install gym-super-mario-bros==7.4.0
!pip install moviepy
!pip install ffmpeg-python==0.2.0

In [85]:
import gym_super_mario_bros
import gym.wrappers
import gym
import cv2
import os
import numpy as np
import glob

from IPython.display import HTML
from IPython.display import Video
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, RIGHT_ONLY, COMPLEX_MOVEMENT
from nes_py.wrappers import JoypadSpace
from base64 import b64encode

In [86]:
class FrameCaptureWrapper(gym.Wrapper):
    def __init__(self, env, video_filename="./mario-rl.mp4", fps=30.0):
        super(FrameCaptureWrapper, self).__init__(env)
        self.frames = []
        self.video_filename = video_filename
        self.fps = fps
        self.video = None

    def _open_video_writer(self):
        first_frame = self.env.render()
        height, width, _ = first_frame.shape
        self.video = cv2.VideoWriter(self.video_filename, cv2.VideoWriter_fourcc(*'mp4v'), self.fps, (width, height))
        #cv2.VideoWriter(output_name + '.avi', cv2.VideoWriter_fourcc('M','J','P','G'), fps, (source[0].shape[1], source[0].shape[0]), True)

    def _close_video_writer(self):
        if self.video is not None:
            self.video.release()

    def reset(self, **kwargs):
        self.frames = []
        self._close_video_writer()
        obs = self.env.reset(**kwargs)
        self._open_video_writer()
        return obs

    def step(self, action):
        if self.video is None:
            raise RuntimeError("Video writer is not initialized. Call reset before stepping.")
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.video.write(cv2.cvtColor(self.env.render(), cv2.COLOR_RGB2BGR))
        return obs, reward, terminated, truncated, info

    def close(self):
        self._close_video_writer()
        super(FrameCaptureWrapper, self).close()

In [89]:
def return_reward(info):
  # reward function that returns rewards/penalties for actions

  reward = 0
  coins, flag_get = info['coins'], info['flag_get']
  time_left, status =  info['time'], info['status']
  x_pos, y_pos = info['x_pos'], info['y_pos']
  # get info for actions

  if coins > coins_total:
    reward += 1
  # increase reward for Mario getting a coin

  if status == 'fireball':
    reward += 2
  elif status == 'tall':
    reward += 1
  else:
    reward += 0
  # increases reward for Mario being more powerful

  if flag_get == True:
    reward += 100
  # increases reward for Mario reaching flag at the end of the level

  if x_pos == prev_x_pos and y_pos == prev_y_pos:
    reward -= 2
  else:
    reward += 1
  # decrease reward for Mario staying still/increase it for moving

  reward  -= (400 - time_left)/400
  # decrease reward for time running out

  return reward


In [90]:
#env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="rgb_array")
env = gym_super_mario_bros.make('SuperMarioBros-v0',
                                apply_api_compatibility=True,
                                render_mode="rgb_array")

env = FrameCaptureWrapper(env, video_filename="./mario-rl.mp4", fps=30)
env.reset()
frames = []
# initalize enviornment and frames
done = False
score = 0
current_time = 400 # start time for game
coins_total = 0  # start number of coins at zero
prev_x_pos, prev_y_pos = 0, 0
# initialize positions at 0 (note: not actual start coordinates)
while current_time > 0:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    coins_total = info['coins']
    current_time = info['time']
    # get current time and current total number of coins
    score+= return_reward(info)
    # add rewards to score
    if current_time == 0 or info['flag_get'] == True or info['life'] == 0:
        done = True
    # end the game if these conditions are met
    done = terminated or truncated
    if done:
       env.reset()
    prev_x_pos, prev_y_pos = info['x_pos'], info['y_pos']
    # save Mario position
env.close()

frames = env.frames
print("\nTotal score: {score}".format(score = score))


Total score: -19183.180000000262


In [68]:
# Define the directory where your files are located
directory = "./"

# Define the file extension you're interested in
file_extension = "*.mp4"

# Use glob to search for files with the specified extension in the directory
file_list = glob.glob(directory + file_extension)

# Iterate through the list of files found
for file_path in file_list:
    # Do whatever you need to do with each file
    print("Found file:", file_path)

Found file: ./mario-rl.mp4
Found file: ./mario-rl-comp.mp4


In [76]:
file_path[0]

'.'

In [77]:
#using embeded video player (jupyter or vscode)
#Video(filename=file_list[0],embed=True)

video_path = file_list[0]
# Embed the video using HTML5 video tag
HTML(f"""
<video width="640" height="480" controls>
  <source src="{video_path}" type="video/mp4">
</video>
""")

In [78]:
os.listdir(os.getcwd())

['.config', 'mario-rl.mp4', 'mario-rl-comp.mp4', 'sample_data']

In [79]:
save_path = 'mario-rl.mp4'
compressed_path = "mario-rl-comp.mp4"

In [80]:
os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [83]:
for key in info.keys():
    print(key, info[key])

coins 0
flag_get False
life 1
score 100
stage 1
status small
time 0
world 1
x_pos 592
y_pos 95
